In [25]:
FILE_PATH = "C:/Ambarish/shopppeedata2/"

# Model Settings
MODEL_NAME="clip-ViT-B-32"

# Pinecone Settings
PINECONE_API_KEY="e72e0b7c-e243-41eb-b497-3fc23b3105fb"
PINECONE_INDEX_NAME="docs"
PINECONE_REGION="us-east1-gcp"
CATEGORY="IMAGES"

In [26]:
from PIL import Image
import pinecone
from sentence_transformers import SentenceTransformer
import os
import pandas as pd

In [27]:
train = pd.read_csv("train.csv")

In [28]:
train.head()

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069


In [29]:
files = os.listdir(FILE_PATH)

In [30]:
df_files = pd.DataFrame(files,columns=["image"])

In [31]:
df_files.head()

,image
0,0000a68812bc7e98c42888dfb1c07da0.jpg
1,000a190fdd715a2a36faed16e2c65df7.jpg
2,00a56fef82c06a746b24237f66f4c6de.jpg
3,00a5b7d85f39222ad75b0d69886a6b58.jpg
4,00a798efae9db3d6d5324daebfea2652.jpg


In [32]:
df_files=df_files.merge(train,on="image",how="inner")

In [33]:
title = df_files["title"].tolist()

In [34]:
len(title),len(files)

(542, 500)

In [35]:
title_all = []
file_name_list = []

In [36]:
for filename in files:
    x = df_files[df_files["image"]==filename]
    for index,row in x.iterrows():
        title_name = ""
        file_name_ = ""
        if (index > 0):
            title_name = title_name + " " + row["title"]
            file_name_ = file_name_ + " " + row["image"]
        else:
            title_name = row["title"]
            file_name_ = row["image"]
    title_all.append(title_name.strip())
    file_name_list.append(file_name_.strip())

In [37]:
len(title_all)

500

In [38]:
title_all[67]

'D060 JTFS KAOS KAKI POLOS KAOS KAKI BAHAN SPANDEK PLUS POLYESTER KAOS KAKI MURAH BATAM IMPORT'

In [40]:
file_name_list[67]

'0a2dfbf3d0a711d5f96451648dbbdac0.jpg'

In [124]:
from pinecone_text.sparse import BM25Encoder

In [125]:
bm25 = BM25Encoder()

In [126]:
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_REGION)
index = pinecone.Index(PINECONE_INDEX_NAME)

In [127]:
model = SentenceTransformer(MODEL_NAME)

In [128]:
bm25.fit(title_all)

  0%|          | 0/501 [00:00<?, ?it/s]

In [129]:
sparse_embeds = bm25.encode_documents(title_all[0])

In [130]:
title_all[171]

'Dear Me Beauty Perfect Lasting Lip Tint Liptint'

In [131]:
len(title_all[171])

47

In [132]:
type(sparse_embeds)

dict

In [133]:
import uuid

In [134]:
def addData(filename,counter):
    id = str(uuid.uuid4())

    index.upsert(vectors=[{
        'id':id, 
        'values':model.encode(Image.open(filename)).tolist(), 
        'metadata':{'category':CATEGORY,'filename':filename},
           'sparse_values':
           bm25.encode_documents(title_all[counter])}],
    )

In [135]:
def insert_pinecone(filename,counter):
    print("Processing file: ", filename)
    addData(filename,counter)

In [136]:
def insert_pinecone_file_path(file_path):
    files = os.listdir(file_path)
    counter = 0
    for filename in files:
        if filename.endswith(".jpg"):
            insert_pinecone(file_path + filename,counter)
            counter = counter + 1

In [137]:
insert_pinecone_file_path(FILE_PATH)

Processing file:  C:/Ambarish/shopppeedata2/0000a68812bc7e98c42888dfb1c07da0.jpg
Processing file:  C:/Ambarish/shopppeedata2/000a190fdd715a2a36faed16e2c65df7.jpg
Processing file:  C:/Ambarish/shopppeedata2/00a56fef82c06a746b24237f66f4c6de.jpg
Processing file:  C:/Ambarish/shopppeedata2/00a5b7d85f39222ad75b0d69886a6b58.jpg
Processing file:  C:/Ambarish/shopppeedata2/00a798efae9db3d6d5324daebfea2652.jpg
Processing file:  C:/Ambarish/shopppeedata2/00a85688de263b6d315d11e1223e15aa.jpg
Processing file:  C:/Ambarish/shopppeedata2/00a866b5d5db175b782db8c01a8e178a.jpg
Processing file:  C:/Ambarish/shopppeedata2/00a8d112066ea78e33d5ef987091a6b4.jpg
Processing file:  C:/Ambarish/shopppeedata2/00b116ee1a2f59352756dceddf542943.jpg
Processing file:  C:/Ambarish/shopppeedata2/00b2375ec81ee1a5c739a62acc74f216.jpg
Processing file:  C:/Ambarish/shopppeedata2/00b28182dd5b6f4615d92bd0a6d31810.jpg
Processing file:  C:/Ambarish/shopppeedata2/00b558da33ffb34b620b0b5e79420c23.jpg
Processing file:  C:/Ambaris